# Test SAFEtorch pipeline

In [ ]:
#ubuntu
import sys 
sys.path.append('/home/mattia/Desktop/tesi_magistrale/SAFEtorch/SAFEtorch')

In [ ]:
from utils.function_normalizer import FunctionNormalizer
from utils.instructions_converter import InstructionsConverter
from utils.capstone_disassembler import disassemble
from utils.radare_analyzer import BinaryAnalyzer
from safetorch.safe_network import SAFE
from safetorch.parameters import Config
import torch

In [ ]:
# initialize SAFE
config = Config()
safe = SAFE(config)

In [ ]:
# load instruction converter and normalizer
#ubuntu
I2V_FILENAME = "../../SAFEtorch/SAFEtorch/model/word2id.json"
#windows
#I2V_FILENAME = "SAFEtorch/model/word2id.json"
converter = InstructionsConverter(I2V_FILENAME)
normalizer = FunctionNormalizer(max_instruction=150)

In [ ]:
# load SAFE weights
#windows
#SAFE_torch_model_path = "SAFEtorch/model/SAFEtorch.pt"
#ubuntu
SAFE_torch_model_path = "../../SAFEtorch/SAFEtorch/model/SAFEtorch.pt"
state_dict = torch.load(SAFE_torch_model_path)
safe.load_state_dict(state_dict)
safe = safe.eval()

# Create exploits_embeddings

In [ ]:
#change these variables according to your setup
#path to folder with exploits
#windows
#exploit_folder = 'C:/Users/matti/Desktop/Magistrale/tesi/poc_detection/temp_dir/'
#ubuntu
exploit_folder = '/media/mattia/2068D30968D2DC9A1/Users/matti/Desktop/Magistrale/tesi/poc_detection/temp_dir/'


#path to embeddings_extractor script
embeddings_extractor = 'embeddings_extractor.py'


In [ ]:
'''
from SAFEtorch.utils.function_normalizer import FunctionNormalizer
from SAFEtorch.utils.instructions_converter import InstructionsConverter
from SAFEtorch.utils.capstone_disassembler import disassemble
from SAFEtorch.utils.radare_analyzer import BinaryAnalyzer
from SAFEtorch.safetorch.safe_network import SAFE
from SAFEtorch.safetorch.parameters import Config
'''
import torch
import gc
gc.collect()
import os
filtered = filter(lambda elem: 'exe' in exploit_folder + elem, os.listdir(exploit_folder))
filtered = map(lambda elem: exploit_folder + elem, filtered)
binary_exploits = list(filtered)

# create dict with {exploit name, (function_name,embedding)}
exploits_embeddings = {}
    
for exe in binary_exploits:
    !python $embeddings_extractor $exe
    exploits_embeddings[exe.replace('.exe','').replace(exploit_folder,'')] = torch.load('/home/mattia/Desktop/tesi_magistrale/SAFEtorch/SAFEtorch/dizionarione.pt')





    '''
    listone = !python $embeddings_extractor $exe
    if len(listone) > 0:
        exploits_embeddings[exe.replace('.exe','').replace(exploit_folder,'')] = listone
        print(listone)
    print(exe.replace('.exe','').replace(exploit_folder,'') + ' processed')
    '''



In [ ]:
torch.save(exploits_embeddings,'/media/mattia/2068D30968D2DC9A1/Users/matti/Desktop/Magistrale/tesi/poc_detection/temp_dir/exploits_embeddings_complete.pt')

In [ ]:
exploits_embeddings = torch.load('/media/mattia/2068D30968D2DC9A1/Users/matti/Desktop/Magistrale/tesi/poc_detection/temp_dir/exploits_embeddings_complete.pt')
print(exploits_embeddings)

## Remove invalid entries in exploits_embeddings

In [ ]:
# remove invalid entries in exploits_embeddings
from copy import deepcopy
d = deepcopy(exploits_embeddings)
# Check if a tensor is zero: if my_tensor.float().sum().data[0] == 0:
for key in d:
    for minikey in d[key]:
        if d[key][minikey].float().sum().item() == 0:
            del exploits_embeddings[key][minikey]
            #print(str(a) + ' ' + str(a==0))


print(exploits_embeddings)
# save exploits_embeddings as file
#torch.save(exploits_embeddings,'/home/mattia/Desktop/tesi_magistrale/SAFEtorch/SAFEtorch/exploits_embeddings_complete.pt')

In [ ]:
#Save clean exploits_embeddings
torch.save(exploits_embeddings,'/media/mattia/2068D30968D2DC9A1/Users/matti/Desktop/Magistrale/tesi/poc_detection/temp_dir/exploits_embeddings_complete.pt')

# Load exploits_embeddings from file

In [ ]:
#windows
#exploits_embeddings = torch.load('C:\\Users\\matti\\Desktop\\Magistrale\\tesi\\poc_detection\\datasets\\exploits_embeddings_complete.pt')
#ubuntu
exploits_embeddings = torch.load('/media/mattia/2068D30968D2DC9A1/Users/matti/Desktop/Magistrale/tesi/poc_detection/temp_dir/exploits_embeddings_complete.pt')
print('exploits_embeddings loaded')

In [ ]:
exploits_embeddings

In [ ]:
def cosine_similarity(fun1,fun2):
    cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
    return cos(fun1, fun2)

def max_similarity(nome, embedding, lista2):
    res = 0
    n1,n2 = ''
    for(nome2,embedding2) in lista2:
        cos = cosine_similarity(embedding,embedding2)
        if(cos > res):
            res = cos
            n1 = nome
            n2 = nome2
    return res,n1,n2

exe = 'C:/Users/matti/Desktop/Magistrale/tesi/poc_detection/temp_dir/10102.exe'
listone = !python $embeddings_extractor $exe
print(listone)